# Adaptive PDE discretizations on Cartesian grids, Taichi implementations

## Volume : Divergence form PDEs

## Part : Scalar wave equation

## Chapter : Velocity-Stress formulation

$
\DeclareMathOperator\diver{div}
\newcommand\<{\langle} \newcommand\>{\rangle}
\newcommand\bZ{\mathbb{Z}}
$

**Hamiltonian description of the wave equation.** The *scalar anisotropic wave equation* is written in position-momentum coordinates as 
\begin{align*}
    \partial_t q &= \mu p, &
    \partial_t p &= \diver(D \nabla q),
\end{align*}
where $\mu$ is the inverse of the medium density, and $D$ is a field of positive definite matrices.
This approach lends itself to an efficient discretization based on symplectic schemes for the system with Hamiltonian 
$$
H(q,p) = \frac 1 2 \int_\Omega \mu p^2 + \<\nabla q,D \nabla q\>,
$$
see the notebooks (**TODO : links**).

**Velocity-Stress formulation.** However, one often also considers the velocity-stress pair of variables, defined as 
\begin{align*}
    v &= \mu p, &
    \sigma &= D \nabla q,
\end{align*}
which is often more convenient for the description of absorbing layers and boundary conditions. In this notebook, we shall discretize the PDE
\begin{align*}
    \partial_t \sigma &= D \nabla v - \alpha \sigma,&
    \partial_t v &= \mu \diver \sigma - \alpha v,
\end{align*}
in a sponge medium with absorption coefficient $\alpha\geq 0$. We use the absorbing boundary condition
$$
    \<\sigma,n\> = -\zeta v,
$$
with parameter $\zeta \geq 0$. The velocity-stress and the position-momentum discretizations are equivalent when $\alpha = \zeta = 0$.

**Decomposition of the anisotropy.**
We assume that 
$$
    D(x) = \sum_{e \in E} \lambda^e(x) e e^\top,
$$
where $E \subset \bZ^d$ is a *fixed* set of offsets. This choice can be questioned: 
- A fixed $E$ yields a simpler implementation, possibly more efficient when anisotropy is mild.
- A variable $E=E(x)$ yields a more complex implementation, especially in the velocity-stress formulation, but is more efficient when anisotropy is strong.

The discrete variables are $v(x)$ and $\sigma(x,e)$, $x \in \Omega_h$, $e\in E$. 

## 0. Importing the required libraries

In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import taichi as ti
ti.init(arch=ti.cpu,default_fp=ti.f64)

[Taichi] version 1.7.2, llvm 15.0.7, commit 0131dce9, osx, python 3.11.11
[Taichi] Starting on arch=arm64


[I 03/27/25 10:39:01.737 5545810] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


## 1. Preprocessing

- Decomposition of the tensor field $D$
- Conversion to use a constant $E$
- Computation of the shifts associated with the offsets of $E$
- Bit mask to know which offsets fall outside.
- Computation of the weights associated with the edges.

In [3]:
help(ti.field)

Help on function field in module taichi.lang.impl:

field(dtype, *args, **kwargs)
    Defines a Taichi field.
    
    A Taichi field can be viewed as an abstract N-dimensional array, hiding away
    the complexity of how its underlying :class:`~taichi.lang.snode.SNode` are
    actually defined. The data in a Taichi field can be directly accessed by
    a Taichi :func:`~taichi.lang.kernel_impl.kernel`.
    
    See also https://docs.taichi-lang.org/docs/field
    
    Args:
        dtype (DataType): data type of the field. Note it can be vector or matrix types as well.
        shape (Union[int, tuple[int]], optional): shape of the field.
        order (str, optional): order of the shape laid out in memory.
        name (str, optional): name of the field.
        offset (Union[int, tuple[int]], optional): offset of the field domain.
        needs_grad (bool, optional): whether this field participates in autodiff (reverse mode)
            and thus needs an adjoint field to store the gra

In [22]:
m = ti.field(dtype=ti.math.vec4,shape=(3,))
shape = (2,3,4) #np.array((2,3,4))
np.cumprod((1,*shape[::-1][:-1]))[::-1]

@ti.kernel
def test_bits():
    for _ in range(1):
        a:ti.i32 = 2
        b:ti.i32 = 3
        print(a | b)
        print(4 >> 1)
        a|=5
        print(a)
test_bits()

#np.cumprod( shape )

3
2
7


In [24]:
m = ti.math.mat2
m.dtype==ti.f64
np.float64(1)

np.float64(1.0)

In [39]:
arr = ti.field(ti.f32,5)
arr.

Help on method copy_from in module taichi.lang.field:

copy_from(other) method of taichi.lang.field.ScalarField instance
    Copies all elements from another field.
    
    The shape of the other field needs to be the same as `self`.
    
    Args:
        other (Field): The source field.



In [35]:
help(ti.template)

Help on class Template in module taichi.types.annotations:

class Template(builtins.object)
 |  Template(tensor=None, dim=None)
 |  
 |  Type annotation for template kernel parameter.
 |  Useful for passing parameters to kernels by reference.
 |  
 |  See also https://docs.taichi-lang.org/docs/meta.
 |  
 |  Args:
 |      tensor (Any): unused
 |      dim (Any): unused
 |  
 |  Example::
 |  
 |      >>> a = 1
 |      >>>
 |      >>> @ti.kernel
 |      >>> def test():
 |      >>>     print(a)
 |      >>>
 |      >>> @ti.kernel
 |      >>> def test_template(a: ti.template()):
 |      >>>     print(a)
 |      >>>
 |      >>> test(a)  # will print 1
 |      >>> test_template(a)  # will also print 1
 |      >>> a = 2
 |      >>> test(a)  # will still print 1
 |      >>> test_template(a)  # will print 2
 |  
 |  Methods defined here:
 |  
 |  __init__(self, tensor=None, dim=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ---------------------------------

In [81]:
type(ti.math.mat2((1.2,0.2),(0.2,1)))

taichi.lang.matrix.Matrix

In [98]:
from types import SimpleNamespace
import copy
t = SimpleNamespace(a=2,b=3)
t2 = copy.copy(t)
t2.__dict__.update({'c':3})

In [100]:
v = ti.math.vec2(0,1)
#v.outer_product(v)
#v.shape
ti.math.sqrt(2)

np.float64(1.4142135623730951)

In [82]:
from types import SimpleNamespace

In [25]:
help(ti.math.vec2)

Help on VectorType in module taichi.lang.matrix object:

class VectorType(MatrixType)
 |  VectorType(n, dtype)
 |  
 |  Method resolution order:
 |      VectorType
 |      MatrixType
 |      taichi.types.compound_types.CompoundType
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, *args)
 |      Return a vector matching the shape and dtype.
 |      
 |      This function will try to convert the input to a `n`-component vector.
 |      
 |      Example::
 |      
 |          >>> vec3 = VectorType(3, float)
 |      
 |          Create from n scalars:
 |      
 |              >>> v = vec3(1, 2, 3)
 |      
 |          Create from a list/tuple of n scalars:
 |      
 |              >>> v = vec3([1, 2, 3])
 |      
 |          Create from a single scalar
 |      
 |              >>> v = vec3(1)
 |  
 |  __init__(self, n, dtype)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  field(self, **kwargs)
 |  
 |  ndarray(self, **kwargs)
 

In [12]:
help(ti.lang.matrix.MatrixType)

Help on class MatrixType in module taichi.lang.matrix:

class MatrixType(taichi.types.compound_types.CompoundType)
 |  MatrixType(n, m, ndim, dtype)
 |  
 |  Method resolution order:
 |      MatrixType
 |      taichi.types.compound_types.CompoundType
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, *args)
 |      Return a matrix matching the shape and dtype.
 |      
 |      This function will try to convert the input to a `n x m` matrix, with n, m being
 |      the number of rows/cols of this matrix type.
 |      
 |      Example::
 |      
 |          >>> mat4x3 = MatrixType(4, 3, float)
 |          >>> mat2x6 = MatrixType(2, 6, float)
 |      
 |          Create from n x m scalars, of a 1d list of n x m scalars:
 |      
 |              >>> m = mat4x3([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
 |              >>> m = mat4x3(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
 |      
 |          Create from n vectors/lists, with each one of dimension m:
 |      
 |   

In [6]:
def mk_velocity_stress_scheme(μ,D,α,absorbing=False):
    """
    Build a taichi implementation of the velocity-stress formulation associated with the model parameters 
    Inputs : 
    - μ : inverse density 
    - D : field of positice definite matrices
    - α : damping coefficient
    - absorbing : wether to use absorbing boundary conditions (otherwise: Neumann b.c.)

    Outputs : 
    ?? Or simply output the result of the Verlet scheme ?
    - step_σ : implements a step of D_t σ = D grad v, followed with a damping step.
    - step_v : implements a step of D_t v = μ div σ, following with a damping step, which includes the effect of absorbing b.c.
    - dictionnary : various additional stuff, including
       - energy : perturbed energy which is guaranteed to decrease along the iterations.
    """
    
    # Decomposition of the tensor field $D$
    # Conversion to use a constant $E$
    # Computation of the shifts associated with the offsets of $E$
    # Bit mask to know which offsets fall outside.
    # Computation of the weights associated with the edges.

    @ti.kernel
    def step_sigma():
        pass
        
    @ti.kernel
    def step_v(): # D_t v = μ div σ - α v
        pass

    @ti.kernel
    def energy(): # Energy is guaranteed to decrease along the iterations
        pass

    return step_sigma,step_v, {'energy' : energy}